In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import random
import os

import yaml
import cv2
import shutil
import tqdm
import glob
import torch
from ultralytics import YOLO

C:\Users\USER\anaconda3\envs\preparation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = os.getcwd()
img_path = os.path.join(os.path.join(os.path.join(path,"img_data"),"img_data"),"colorful_fashion_dataset_for_object_detection")
images_path = os.path.join(img_path,"JPEGImages")
annotations_path  = os.path.join(img_path,"Annotations_txt")

In [3]:
train_num_path = os.path.join(os.path.join(os.path.join(img_path,"ImageSets"),"Main"), "trainval.txt")
test_num_path = os.path.join(os.path.join(os.path.join(img_path,"ImageSets"),"Main"), "test.txt")

In [4]:
with open(train_num_path,'r') as f:
    train_num = f.read()
with open(test_num_path,'r') as f:
    test_num = f.read()
train_num_list = train_num.split("\n")[:-1]
test_num_list = test_num.split("\n")[:-1]
# print(train_num_list)
# print(test_num_list)

In [5]:
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None
def img_preprocessing(img_path):
    # 한글 경로로 되어 있지 않을 때 1번, 한글 경로로 되어 있다면 2번을 사용함.
    # 1번
    #re_img = cv2.imread(img_path)
    # 2번
    img = imread(img_path)
    
    #전처리 부분
    # 여기서 416,416으로 이미지 크기를 조정함.
    # 이밖에 모자이크, 흑백 등의 전처리를 추가하고 싶다면 자유롭게 할 것.
    re_img = cv2.resize(img, (256, 256))
    return re_img
# 파일 위치가 한글 경로가 존재할 때 사용함(없다면 제외하고 해도 상관 없음)
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)
        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False

In [6]:
def make_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [7]:
# image 가져오기
# image 분리하기
# image 저장하기
train_dir = os.path.join(img_path,"train")
val_dir = os.path.join(img_path,"val")
train_img_dir = os.path.join(train_dir,"images")
val_img_dir = os.path.join(val_dir,"images")
train_label_dir = os.path.join(train_dir,"labels")
val_label_dir = os.path.join(val_dir,"labels")
make_dir(train_dir)
make_dir(val_dir)
make_dir(train_img_dir)
make_dir(val_img_dir)
make_dir(train_label_dir)
make_dir(val_label_dir)

In [8]:
train_img_dir

'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\img_data\\colorful_fashion_dataset_for_object_detection\\train\\images'

In [9]:
for i in train_num_list:
    img_dir = os.path.join(images_path, i+".jpg")
    New_img_dir = os.path.join(train_img_dir,i+".jpg")
    img = imread(img_dir)
    imwrite(New_img_dir,img)
for i in test_num_list:
    img_dir = os.path.join(images_path, i+".jpg")
    New_img_dir = os.path.join(val_img_dir,i+".jpg")
    img = imread(img_dir)
    imwrite(New_img_dir,img)

In [10]:
# 나눠서 저장하기
# 라벨 읽고 저장
for file in train_num_list:
    label_path = os.path.join(annotations_path, file+".txt")
    T_label_path = os.path.join(train_label_dir, file+".txt")
    f = open(label_path, 'r')
    h = open(T_label_path,'w')
    data = f.read()
    h.write(data)
    f.close()
    h.close()
    
for file in test_num_list:
    label_path = os.path.join(annotations_path, file+".txt")
    T_label_path = os.path.join(val_label_dir, file+".txt")
    f = open(label_path, 'r')
    h = open(T_label_path,'w')
    data = f.read()
    h.write(data)
    f.close()
    h.close()

In [11]:
with open(os.path.join(img_path,"labels.txt"),'r') as f:
    data = f.read()
    classes = data
names = classes.split(",\n")
nc = len(names)

In [12]:
yaml_data = {'train': train_img_dir, 'val': val_img_dir, 'nc': nc, 'names': names}
yaml_path = os.path.join(path,'data.yaml')
with open(yaml_path,'w') as f:
    yaml.dump(yaml_data,f)
# 잘 저장되었는 지 확인하기
with open(yaml_path,'r') as f:
    yaml_file = yaml.load(f, Loader=yaml.FullLoader)
    display(yaml_file)


{'names': ['sunglass',
  'hat',
  'jacket',
  'shirt',
  'pants',
  'shorts',
  'skirt',
  'dress',
  'bag',
  'shoe'],
 'nc': 10,
 'train': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\img_data\\colorful_fashion_dataset_for_object_detection\\train\\images',
 'val': 'C:\\Users\\USER\\Desktop\\수\\인공지능 데브코스 6기\\최종 프로젝트\\code\\image code\\img_data\\img_data\\colorful_fashion_dataset_for_object_detection\\val\\images'}